In [1]:
%run "contant.py"

已加载全局配置

🎯 模型常量:
  OPENAI_MODEL, open_model

🎨 打印方法:
  pretty_print, print_json
  print_agent


## pydantic

In [12]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    """人员联系信息。"""
    name: str = Field(description="人员姓名")
    email: str = Field(description="电子邮件地址")
    phone: str = Field(description="电话号码")

agent = create_agent(
    OPENAI_MODEL,
    response_format=ContactInfo  
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "从以下信息提取联系人信息：陈同学, 121543259@qq.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='陈同学', email='121543259@qq.com', phone='(555) 123-4567')

## Dataclasses

In [16]:
from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """联系人信息。"""
    name: str  # 联系人姓名
    email: str  # 联系人电子邮件地址
    phone: str  # 联系人电话号码

agent = create_agent(
    OPENAI_MODEL,
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "从以下信息提取联系人信息：陈同学, 121543259@qq.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='陈同学', email='121543259@qq.com', phone='(555) 123-4567')

## TypedDict

In [15]:
from typing_extensions import TypedDict
from langchain.agents import create_agent

class ContactInfo(TypedDict):
    """联系人信息。"""
    name: str  # 联系人姓名
    email: str  # 联系人电子邮件地址
    phone: str  # 联系人电话号码

agent = create_agent(
    OPENAI_MODEL,
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "从以下信息提取联系人信息：陈同学, 121543259@qq.com, (555) 123-4567"}]
})

result["structured_response"]

{'name': '陈同学', 'email': '121543259@qq.com', 'phone': '(555) 123-4567'}

In [22]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ProductReview(BaseModel):
  """产品评论分析。"""
  rating: int | None = Field(description="产品评分", ge=1, le=5)
  sentiment: Literal["positive", "negative"] = Field(description="评论情感")
  key_points: list[str] = Field(description="评论要点")

agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
  "messages": [{"role": "user", "content": "分析评论：'产品很棒：5星。快递很快，但价格有点贵'"}]
})

result["structured_response"]

ProductReview(rating=5, sentiment='positive', key_points=['产品很棒'])

In [26]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    """联系人信息"""
    name: str = Field(description="人员姓名")
    email: str = Field(description="电子邮件地址")

class EventDetails(BaseModel):
    """活动详情"""
    event_name: str = Field(description="活动名称")
    date: str = Field(description="活动日期")

agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])  # 默认：handle_errors=True
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "在组织 3 月 15 日的技术会议"}]
})

print_json(result)

{
  "messages": [
    {
      "content": "在组织 3 月 15 日的技术会议",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "4316cd80-b0a8-4a23-80ef-093f5b734652"
    },
    {
      "content": "",
      "additional_kwargs": {
        "refusal": null
      },
      "response_metadata": {
        "token_usage": {
          "completion_tokens": 26,
          "prompt_tokens": 98,
          "total_tokens": 124,
          "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
          },
          "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0,
            "cache_creation_input_tokens": 0,
            "cache_read_input_tokens": 0
          }
        },
        "model_provider": "openai",
        "model_name": "gpt-4.1-nano",
        "system_fingerprint": "fp_03e44fcc

## 自定义工具消息内容 (Custom tool message content)

In [31]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class MeetingAction(BaseModel):
    """从会议记录中提取的行动项目。"""
    task: str = Field(description="要完成的具体任务")
    assignee: str = Field(description="负责任务的人员")
    priority: Literal["low", "medium", "high"] = Field(description="优先级")

agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(
        schema=MeetingAction,
        tool_message_content="行动项目已捕获并添加到会议记录中！"   # 插入一条 ToolMessage
    )
)

contact_result = agent.invoke({
    "messages": [{"role": "user", "content": "从我们的会议中：Sarah 需要尽快更新项目时间表"}]
})

print_json(contact_result)

{
  "messages": [
    {
      "content": "从我们的会议中：Sarah 需要尽快更新项目时间表",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "bf5663ea-35e4-48e5-a5dc-a188a418fb85"
    },
    {
      "content": "",
      "additional_kwargs": {
        "refusal": null
      },
      "response_metadata": {
        "token_usage": {
          "completion_tokens": 30,
          "prompt_tokens": 97,
          "total_tokens": 127,
          "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
          },
          "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0,
            "cache_creation_input_tokens": 0,
            "cache_read_input_tokens": 0
          }
        },
        "model_provider": "openai",
        "model_name": "gpt-4.1-nano",
        "system_fingerprint": "fp_

## Union types 支持多种结构化输出（只输出命中率高的）

In [33]:
from typing import Union
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    """联系人信息"""
    name: str = Field(description="人员姓名")
    email: str = Field(description="电子邮件地址")

class EventDetails(BaseModel):
    """活动详情"""
    event_name: str = Field(description="活动名称")
    date: str = Field(description="活动日期")

agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])
)

# 测试联系人信息提取
contact_result = agent.invoke({
    "messages": [{"role": "user", "content": "请提取联系人信息：李明, liming@example.com"}]
})
print("联系人结果:", contact_result["structured_response"])

# 测试活动信息提取
event_result = agent.invoke({
    "messages": [{"role": "user", "content": "请提取活动详情：年度大会，2024年12月20日"}]
})
print("活动结果:", event_result["structured_response"])

联系人结果: name='李明' email='liming@example.com'
活动结果: event_name='年度大会' date='2024-12-20'


In [40]:
from typing import Union
from pydantic import Field
from langchain.agents import create_agent
from langchain.agents.structured_output import MultipleStructuredOutputsError, StructuredOutputValidationError, ToolStrategy

class ContactInfo(BaseModel):
    """联系人信息"""
    name: str = Field(description="人员姓名")
    email: str = Field(description="电子邮件地址")

class EventDetails(BaseModel):
    """活动详情"""
    event_name: str = Field(description="活动名称")
    date: str = Field(description="活动日期")

def custom_error_handler(error: Exception) -> str:
    if isinstance(error, StructuredOutputValidationError):
        return "格式有问题，请重试。"
    elif isinstance(error, MultipleStructuredOutputsError):
        return "返回了多个结构化输出，请选择最相关的一个。"
    else:
        return f"错误: {str(error)}"

agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(
        schema=Union[ContactInfo, EventDetails],
        handle_errors=custom_error_handler
    )  # 默认：handle_errors=True
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "提取信息：John Doe (john@email.com) 正在组织 3 月 15 日的技术会议"}]
})


print_json(result)

# for msg in result['messages']:
#     # 如果消息实际上是 ToolMessage 对象（不是字典），检查其类名
#     if type(msg).__name__ == "ToolMessage":
#         print(msg.content)
#     # 如果消息是字典或者您需要备用方案
#     elif isinstance(msg, dict) and msg.get('tool_call_id'):
#         print(msg['content'])

{
  "messages": [
    {
      "content": "提取信息：John Doe (john@email.com) 正在组织 3 月 15 日的技术会议",
      "additional_kwargs": {},
      "response_metadata": {},
      "type": "human",
      "name": null,
      "id": "3b09aafb-1d1c-4f0a-bda3-8841629e5f00"
    },
    {
      "content": "",
      "additional_kwargs": {
        "refusal": null
      },
      "response_metadata": {
        "token_usage": {
          "completion_tokens": 63,
          "prompt_tokens": 110,
          "total_tokens": 173,
          "completion_tokens_details": {
            "accepted_prediction_tokens": 0,
            "audio_tokens": 0,
            "reasoning_tokens": 0,
            "rejected_prediction_tokens": 0
          },
          "prompt_tokens_details": {
            "audio_tokens": 0,
            "cached_tokens": 0,
            "cache_creation_input_tokens": 0,
            "cache_read_input_tokens": 0
          }
        },
        "model_provider": "openai",
        "model_name": "gpt-4.1-nano",
        "

## 返回多个 Struct 

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class UserInfo(BaseModel):
    """用户信息"""
    name: str = Field(description="用户姓名")
    email: str = Field(description="邮箱地址")

class ProductInfo(BaseModel):
    """产品信息"""
    product_name: str = Field(description="产品名称")
    price: float = Field(description="价格")

class MultiOutput(BaseModel):
    """多结构体输出包装类"""
    users: List[UserInfo] = Field(description="用户信息列表")
    products: List[ProductInfo] = Field(description="产品信息列表")
    summary: str = Field(description="提取总结")

# 创建Agent
agent = create_agent(
    OPENAI_MODEL,
    response_format=ToolStrategy(MultiOutput)
)

# 测试多结构体提取
result = agent.invoke({
    "messages": [{"role": "user", "content": """
    请提取以下信息：
    用户：张三(zhang@example.com)，李四(li@example.com)
    产品：iPhone(¥5999)，MacBook(¥9999)
    """}]
})

print_json(result["structured_response"], "多结构体提取结果")


 多结构体提取结果
{
  "users": [
    {
      "name": "张三",
      "email": "zhang@example.com"
    },
    {
      "name": "李四",
      "email": "li@example.com"
    }
  ],
  "products": [
    {
      "product_name": "iPhone",
      "price": 5999.0
    },
    {
      "product_name": "MacBook",
      "price": 9999.0
    }
  ],
  "summary": "用户：张三(zhang@example.com)，李四(li@example.com)"
}


: 